In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

Mounted at /content/drive


In [ ]:
# @title Function
# @title Function
def kategori_dokument (jumlah):
  if jumlah == 0:
    return 'empty'
  elif jumlah > 10 :
    return 'complate'
  else:
    return 'incomplate'

def information_flag (jumlah):
  if jumlah >= 1 :
    return '1'
  else:
    return '0'

def drop_kolom (data, treshold):
  missing = (data.isnull().sum()/len(data))* 100
  missing_cols = missing[missing > treshold].index
  data.drop(columns = missing_cols, inplace= True)
  return data



In [ ]:
credit_card_balance = pd.read_csv('/content/drive/MyDrive/Colab minipro 4 /Permodelan data Home Credit Indonesia/Data HCI/credit_card_balance.csv')
installment = pd.read_csv('/content/drive/MyDrive/Colab minipro 4 /Permodelan data Home Credit Indonesia/Data HCI/installments_payments.csv')
previous_app = pd.read_csv('/content/drive/MyDrive/Colab minipro 4 /Permodelan data Home Credit Indonesia/Data HCI/previous_application.csv')
posh_cash_balance = pd.read_csv('/content/drive/MyDrive/Colab minipro 4 /Permodelan data Home Credit Indonesia/Data HCI/POS_CASH_balance.csv')

In [ ]:
# @title Instalment Payment
print('Jumlah Data Bureau Balance yang dimiliki: ',installment.shape)
print(f'Jumlah duplikat SK_ID_PREV: ',{installment['SK_ID_PREV'].duplicated().sum()})
installment.sort_values(by='SK_ID_PREV').head(10)

Jumlah Data Bureau Balance yang dimiliki:  (13605401, 8)
Jumlah duplikat SK_ID_PREV:  {12607649}


,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
512588,1000001,158271,1.0,1,-268.0,-294.0,6404.310,6404.310
2159480,1000001,158271,2.0,2,-238.0,-244.0,62039.115,62039.115
3411021,1000002,101962,1.0,3,-1540.0,-1559.0,6264.000,6264.000
1214732,1000002,101962,1.0,1,-1600.0,-1611.0,6264.000,6264.000
1006868,1000002,101962,2.0,4,-1510.0,-1554.0,18443.565,18443.565
1631862,1000002,101962,1.0,2,-1570.0,-1575.0,6264.000,6264.000
7054246,1000003,252457,1.0,2,-64.0,-81.0,4951.350,4951.350
6145031,1000003,252457,1.0,1,-94.0,-108.0,4951.350,4951.350
4279518,1000003,252457,1.0,3,-34.0,-49.0,4951.350,4951.350
5197652,1000004,260094,1.0,5,-742.0,-752.0,3391.110,3391.110


In [ ]:
# menghandle data duplikat
'''Alasan saya tidak melakukan drop duplikat karena data duplikat yang saya miliki memiliki feature number instalment version dengan number instalment number
maka saya memilih untuk melakukan grouping dengan menjumlahkan setiap baris berdasarkan key yaitu (SK_ID_PREV dan SK_ID_CURR) agar value data tidak
banyak yang hilang akibat penggunaan drop duplikat.Kemudian saya akan menghapus kedua feature yang saya sebut sebelumnya dikarenakan sudah tidak dipelukan lagi.'''

instal = installment.groupby(['SK_ID_PREV','SK_ID_CURR']).sum().reset_index()
instal.drop(columns=['NUM_INSTALMENT_VERSION','NUM_INSTALMENT_NUMBER'], inplace=True)
instal.head()

,SK_ID_PREV,SK_ID_CURR,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1000001,158271,-506.0,-538.0,68443.425,68443.425
1,1000002,101962,-6220.0,-6299.0,37235.565,37235.565
2,1000003,252457,-192.0,-238.0,14854.050,14854.050
3,1000004,260094,-5404.0,-5591.0,33523.155,33523.155
4,1000005,176456,-16978.0,-17071.0,161735.310,147021.705


In [ ]:
print('Total Missing:\n',instal.isnull().sum())
print('Sisa Data setelah dilakukan Dropna: ',instal.shape)
instal.columns

Total Missing:
 SK_ID_PREV            0
SK_ID_CURR            0
DAYS_INSTALMENT       0
DAYS_ENTRY_PAYMENT    0
AMT_INSTALMENT        0
AMT_PAYMENT           0
dtype: int64
Sisa Data setelah dilakukan Dropna:  (997752, 6)


Index(['SK_ID_PREV', 'SK_ID_CURR', 'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT',
       'AMT_INSTALMENT', 'AMT_PAYMENT'],
      dtype='object')

In [ ]:
# @title Tabel Credit Card Balance
print('Total Data yang dimiliki: ',credit_card_balance.shape)
print(f'Jumlah duplikat SK_ID_PREV: ',{credit_card_balance['SK_ID_PREV'].duplicated().sum()})
credit_card_balance.sort_values(by='SK_ID_PREV').head(10)

Total Data yang dimiliki:  (3840312, 23)
Jumlah duplikat SK_ID_PREV:  {3736005}


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
2159094,1000018,394447,-2,136695.420,135000,0.0,22827.330,0.0,22827.330,6206.67,7970.535,7970.535,132903.000,136024.920,136024.920,0.0,11,0.0,11.0,4.0,Active,0,0
2559582,1000018,394447,-4,44360.505,45000,0.0,2032.560,0.0,2032.560,2250.00,3646.710,3646.710,43376.760,44360.505,44360.505,0.0,2,0.0,2.0,2.0,Active,0,0
277653,1000018,394447,-5,40934.070,45000,0.0,2335.500,0.0,2335.500,2250.00,3900.870,3900.870,39576.780,40934.070,40934.070,0.0,2,0.0,2.0,1.0,Active,0,0
2192275,1000018,394447,-3,113862.285,135000,13500.0,69156.945,0.0,55656.945,2263.77,3190.635,3190.635,108091.800,109150.785,109150.785,3.0,14,0.0,11.0,3.0,Active,0,0
1375415,1000018,394447,-6,38879.145,45000,13500.0,51042.645,0.0,37542.645,0.00,9000.000,9000.000,37542.645,37542.645,37542.645,3.0,15,0.0,12.0,0.0,Active,0,0
866526,1000030,361282,-3,100264.455,112500,0.0,46660.500,0.0,46660.500,2250.00,4849.650,322.380,99775.260,100264.455,100264.455,0.0,4,0.0,4.0,3.0,Active,0,0
1884819,1000030,361282,-8,0.000,45000,NaN,0.000,NaN,NaN,0.00,NaN,0.000,0.000,0.000,0.000,NaN,0,NaN,NaN,0.0,Active,0,0
1597893,1000030,361282,-2,99195.075,112500,0.0,1849.050,0.0,1849.050,4527.27,6442.695,1094.175,98019.090,99195.075,99195.075,0.0,2,0.0,2.0,4.0,Active,0,0
1150224,1000030,361282,-1,103027.275,112500,0.0,6550.650,0.0,6550.650,5348.52,5998.590,567.540,101866.725,103027.275,103027.275,0.0,3,0.0,3.0,5.0,Active,0,0
2192311,1000030,361282,-6,33784.740,45000,0.0,20212.650,0.0,20212.650,0.00,2567.655,317.655,33725.745,33784.740,33784.740,0.0,8,0.0,8.0,0.0,Active,0,0


In [ ]:
credit_card = credit_card_balance.add_suffix('_card')
credit_card.rename(columns={'SK_ID_PREV_card':'SK_ID_PREV'},inplace=True)

In [ ]:
# menghandle data duplicat
''' Saya akan melakukan hal yang sama seperti pada sebelumnya, penyebab data yang saya miliki memiliki key duplikat dikarenakan terdapat feature Month balance
oleh sebab itu saya akan lakukan hal yang sama.'''
credit = credit_card.groupby(['SK_ID_PREV','SK_ID_CURR_card','NAME_CONTRACT_STATUS_card']).sum().reset_index()
credit.drop(columns=['MONTHS_BALANCE_card'], inplace=True)

# menampilkan jumlah missing data
print('Sisa data yang dimiliki',credit.shape)
print('Data Missing= ',credit.isnull().sum()/len(credit)*100)
credit.head()

Sisa data yang dimiliki (122988, 22)
Data Missing=  SK_ID_PREV                         0.0
SK_ID_CURR_card                    0.0
NAME_CONTRACT_STATUS_card          0.0
AMT_BALANCE_card                   0.0
AMT_CREDIT_LIMIT_ACTUAL_card       0.0
AMT_DRAWINGS_ATM_CURRENT_card      0.0
AMT_DRAWINGS_CURRENT_card          0.0
AMT_DRAWINGS_OTHER_CURRENT_card    0.0
AMT_DRAWINGS_POS_CURRENT_card      0.0
AMT_INST_MIN_REGULARITY_card       0.0
AMT_PAYMENT_CURRENT_card           0.0
AMT_PAYMENT_TOTAL_CURRENT_card     0.0
AMT_RECEIVABLE_PRINCIPAL_card      0.0
AMT_RECIVABLE_card                 0.0
AMT_TOTAL_RECEIVABLE_card          0.0
CNT_DRAWINGS_ATM_CURRENT_card      0.0
CNT_DRAWINGS_CURRENT_card          0.0
CNT_DRAWINGS_OTHER_CURRENT_card    0.0
CNT_DRAWINGS_POS_CURRENT_card      0.0
CNT_INSTALMENT_MATURE_CUM_card     0.0
SK_DPD_card                        0.0
SK_DPD_DEF_card                    0.0
dtype: float64


,SK_ID_PREV,SK_ID_CURR_card,NAME_CONTRACT_STATUS_card,AMT_BALANCE_card,AMT_CREDIT_LIMIT_ACTUAL_card,AMT_DRAWINGS_ATM_CURRENT_card,AMT_DRAWINGS_CURRENT_card,AMT_DRAWINGS_OTHER_CURRENT_card,AMT_DRAWINGS_POS_CURRENT_card,AMT_INST_MIN_REGULARITY_card,AMT_PAYMENT_CURRENT_card,AMT_PAYMENT_TOTAL_CURRENT_card,AMT_RECEIVABLE_PRINCIPAL_card,AMT_RECIVABLE_card,AMT_TOTAL_RECEIVABLE_card,CNT_DRAWINGS_ATM_CURRENT_card,CNT_DRAWINGS_CURRENT_card,CNT_DRAWINGS_OTHER_CURRENT_card,CNT_DRAWINGS_POS_CURRENT_card,CNT_INSTALMENT_MATURE_CUM_card,SK_DPD_card,SK_DPD_DEF_card
0,1000018,394447,Active,374731.425,405000,27000.0,147394.980,0.0,120394.980,12970.44,27708.75,27708.75,361490.985,368012.925,368012.925,6.0,44,0.0,38.0,10.0,0,0
1,1000030,361282,Active,447928.515,652500,4500.0,138059.505,0.0,133559.505,16625.79,43320.42,21263.58,443795.625,447483.015,447483.015,1.0,41,0.0,40.0,15.0,0,0
2,1000031,131335,Active,838311.030,2394000,157500.0,463353.840,0.0,305853.840,42804.81,354519.09,354519.09,822446.055,833599.530,833599.530,4.0,21,0.0,17.0,59.0,0,0
3,1000035,436351,Active,0.000,1125000,0.0,0.000,0.0,0.000,0.00,0.00,0.00,0.000,0.000,0.000,0.0,0,0.0,0.0,0.0,0,0
4,1000077,181153,Active,0.000,1035000,0.0,0.000,0.0,0.000,0.00,0.00,0.00,0.000,0.000,0.000,0.0,0,0.0,0.0,0.0,0,0


In [ ]:
# @title Join table
card = pd.merge(instal, credit, how='inner', on='SK_ID_PREV')
print('Total data: ',card.shape)
card.head()

Total data:  (85804, 27)


,SK_ID_PREV,SK_ID_CURR,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,SK_ID_CURR_card,NAME_CONTRACT_STATUS_card,AMT_BALANCE_card,AMT_CREDIT_LIMIT_ACTUAL_card,AMT_DRAWINGS_ATM_CURRENT_card,AMT_DRAWINGS_CURRENT_card,AMT_DRAWINGS_OTHER_CURRENT_card,AMT_DRAWINGS_POS_CURRENT_card,AMT_INST_MIN_REGULARITY_card,AMT_PAYMENT_CURRENT_card,AMT_PAYMENT_TOTAL_CURRENT_card,AMT_RECEIVABLE_PRINCIPAL_card,AMT_RECIVABLE_card,AMT_TOTAL_RECEIVABLE_card,CNT_DRAWINGS_ATM_CURRENT_card,CNT_DRAWINGS_CURRENT_card,CNT_DRAWINGS_OTHER_CURRENT_card,CNT_DRAWINGS_POS_CURRENT_card,CNT_INSTALMENT_MATURE_CUM_card,SK_DPD_card,SK_DPD_DEF_card
0,1000018,394447,-1332.0,-1338.0,35257.905,35257.905,394447,Active,374731.425,405000,27000.0,147394.980,0.0,120394.980,12970.440,27708.750,27708.750,361490.985,368012.925,368012.925,6.0,44,0.0,38.0,10.0,0,0
1,1000030,361282,-2201.0,-2245.0,43398.495,43398.495,361282,Active,447928.515,652500,4500.0,138059.505,0.0,133559.505,16625.790,43320.420,21263.580,443795.625,447483.015,447483.015,1.0,41,0.0,40.0,15.0,0,0
2,1000031,131335,-6419.0,-6542.0,354519.090,354519.090,131335,Active,838311.030,2394000,157500.0,463353.840,0.0,305853.840,42804.810,354519.090,354519.090,822446.055,833599.530,833599.530,4.0,21,0.0,17.0,59.0,0,0
3,1000087,399664,-23824.0,-23876.0,202900.725,197451.360,399664,Active,1250499.330,2295000,108000.0,136911.195,0.0,28911.195,67284.225,197287.425,197287.425,1207948.815,1250733.060,1250733.060,7.0,13,0.0,6.0,275.0,0,0
4,1000094,359175,-160176.0,-160582.0,269075.205,260075.205,359175,Active,2583953.145,4522500,138600.0,138601.035,0.0,0.000,169480.665,260075.205,256591.035,2466087.210,2583492.480,2586012.480,8.0,8,0.0,0.0,2574.0,1,1


In [ ]:
# @title Tabel Posh Cash Balance
print('Total Data yang dimiliki: ',posh_cash_balance.shape)
print(f'Jumlah duplikat SK_ID_PREV: ',{posh_cash_balance['SK_ID_PREV'].duplicated().sum()})
posh_cash_balance.sort_values(by='SK_ID_CURR').head(10)

Total Data yang dimiliki:  (10001358, 8)
Jumlah duplikat SK_ID_PREV:  {9065033}


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
8531326,1851984,100001,-94,4.0,0.0,Active,0,0
1891462,1851984,100001,-95,4.0,1.0,Active,7,7
8789081,1369693,100001,-56,4.0,3.0,Active,0,0
4704415,1369693,100001,-54,4.0,1.0,Active,0,0
7167007,1369693,100001,-57,4.0,4.0,Active,0,0
4928574,1851984,100001,-93,4.0,0.0,Completed,0,0
1261679,1851984,100001,-96,4.0,2.0,Active,0,0
7823681,1369693,100001,-55,4.0,2.0,Active,0,0
2197888,1369693,100001,-53,4.0,0.0,Completed,0,0
9849444,1038818,100002,-15,24.0,20.0,Active,0,0


In [ ]:
posh_cash = posh_cash_balance.add_suffix('_pos')
posh_cash.rename(columns={'SK_ID_CURR_pos':'SK_ID_CURR'},inplace=True)

# menghandle data duplicat
''' Saya akan melakukan hal yang sama seperti pada sebelumnya, penyebab data yang saya miliki memiliki key duplikat dikarenakan terdapat feature Month balance
oleh sebab itu saya akan lakukan hal yang sama.'''
posh = posh_cash.groupby(['SK_ID_PREV_pos','SK_ID_CURR','NAME_CONTRACT_STATUS_pos']).sum().reset_index()
posh.drop(columns=['MONTHS_BALANCE_pos'], inplace=True)


# menampilkan jumlah missing data
print('Sisa data yang dimiliki',posh.shape)
print('Data Missing= ',posh.isnull().sum()/len(credit)*100)
posh.head()

Sisa data yang dimiliki (1719663, 7)
Data Missing=  SK_ID_PREV_pos               0.0
SK_ID_CURR                   0.0
NAME_CONTRACT_STATUS_pos     0.0
CNT_INSTALMENT_pos           0.0
CNT_INSTALMENT_FUTURE_pos    0.0
SK_DPD_pos                   0.0
SK_DPD_DEF_pos               0.0
dtype: float64


,SK_ID_PREV_pos,SK_ID_CURR,NAME_CONTRACT_STATUS_pos,CNT_INSTALMENT_pos,CNT_INSTALMENT_FUTURE_pos,SK_DPD_pos,SK_DPD_DEF_pos
0,1000001,158271,Active,24.0,23.0,0,0
1,1000001,158271,Completed,2.0,0.0,0,0
2,1000002,101962,Active,22.0,10.0,0,0
3,1000002,101962,Completed,4.0,0.0,0,0
4,1000003,252457,Active,48.0,42.0,0,0


In [ ]:
# @title Join table
posh_card = pd.merge(card, posh, how='inner',on='SK_ID_CURR')
print('Jumlah Data: ',posh_card.shape)
posh_card.head()

Jumlah Data:  (518011, 33)


,SK_ID_PREV,SK_ID_CURR,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,SK_ID_CURR_card,NAME_CONTRACT_STATUS_card,AMT_BALANCE_card,AMT_CREDIT_LIMIT_ACTUAL_card,AMT_DRAWINGS_ATM_CURRENT_card,AMT_DRAWINGS_CURRENT_card,AMT_DRAWINGS_OTHER_CURRENT_card,AMT_DRAWINGS_POS_CURRENT_card,AMT_INST_MIN_REGULARITY_card,AMT_PAYMENT_CURRENT_card,AMT_PAYMENT_TOTAL_CURRENT_card,AMT_RECEIVABLE_PRINCIPAL_card,AMT_RECIVABLE_card,AMT_TOTAL_RECEIVABLE_card,CNT_DRAWINGS_ATM_CURRENT_card,CNT_DRAWINGS_CURRENT_card,CNT_DRAWINGS_OTHER_CURRENT_card,CNT_DRAWINGS_POS_CURRENT_card,CNT_INSTALMENT_MATURE_CUM_card,SK_DPD_card,SK_DPD_DEF_card,SK_ID_PREV_pos,NAME_CONTRACT_STATUS_pos,CNT_INSTALMENT_pos,CNT_INSTALMENT_FUTURE_pos,SK_DPD_pos,SK_DPD_DEF_pos
0,1000018,394447,-1332.0,-1338.0,35257.905,35257.905,394447,Active,374731.425,405000,27000.0,147394.980,0.0,120394.980,12970.44,27708.75,27708.75,361490.985,368012.925,368012.925,6.0,44,0.0,38.0,10.0,0,0,1210150,Active,30.0,20.0,0,0
1,1000030,361282,-2201.0,-2245.0,43398.495,43398.495,361282,Active,447928.515,652500,4500.0,138059.505,0.0,133559.505,16625.79,43320.42,21263.58,443795.625,447483.015,447483.015,1.0,41,0.0,40.0,15.0,0,0,1944208,Active,30.0,20.0,0,0
2,1000030,361282,-2201.0,-2245.0,43398.495,43398.495,361282,Active,447928.515,652500,4500.0,138059.505,0.0,133559.505,16625.79,43320.42,21263.58,443795.625,447483.015,447483.015,1.0,41,0.0,40.0,15.0,0,0,1944208,Completed,5.0,0.0,0,0
3,1000031,131335,-6419.0,-6542.0,354519.090,354519.090,131335,Active,838311.030,2394000,157500.0,463353.840,0.0,305853.840,42804.81,354519.09,354519.09,822446.055,833599.530,833599.530,4.0,21,0.0,17.0,59.0,0,0,1312860,Active,30.0,27.0,0,0
4,1000031,131335,-6419.0,-6542.0,354519.090,354519.090,131335,Active,838311.030,2394000,157500.0,463353.840,0.0,305853.840,42804.81,354519.09,354519.09,822446.055,833599.530,833599.530,4.0,21,0.0,17.0,59.0,0,0,2048100,Active,144.0,78.0,0,0


In [ ]:
# @title Tabel Previous App
print('Total Data yang dimiliki: ',previous_app.shape)
print(f'Jumlah duplikat SK_ID_PREV: ',{previous_app['SK_ID_PREV'].duplicated().sum()})
previous_app.sort_values(by='SK_ID_CURR').head(5)

Total Data yang dimiliki:  (1670214, 37)
Jumlah duplikat SK_ID_PREV:  {0}


,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
201668,1369693,100001,Consumer loans,3951.000,24835.5,23787.0,2520.0,24835.5,FRIDAY,13,Y,1,0.104326,NaN,NaN,XAP,Approved,-1740,Cash through the bank,XAP,Family,Refreshed,Mobile,POS,XNA,Country-wide,23,Connectivity,8.0,high,POS mobile with interest,365243.0,-1709.0,-1499.0,-1619.0,-1612.0,0.0
892077,1038818,100002,Consumer loans,9251.775,179055.0,179055.0,0.0,179055.0,SATURDAY,9,Y,1,0.000000,NaN,NaN,XAP,Approved,-606,XNA,XAP,NaN,New,Vehicles,POS,XNA,Stone,500,Auto technology,24.0,low_normal,POS other with interest,365243.0,-565.0,125.0,-25.0,-17.0,0.0
575941,1810518,100003,Cash loans,98356.995,900000.0,1035882.0,NaN,900000.0,FRIDAY,12,Y,1,NaN,NaN,NaN,XNA,Approved,-746,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,low_normal,Cash X-Sell: low,365243.0,-716.0,-386.0,-536.0,-527.0,1.0
1021650,2636178,100003,Consumer loans,64567.665,337500.0,348637.5,0.0,337500.0,SUNDAY,17,Y,1,0.000000,NaN,NaN,XAP,Approved,-828,Cash through the bank,XAP,Family,Refreshed,Furniture,POS,XNA,Stone,1400,Furniture,6.0,middle,POS industry with interest,365243.0,-797.0,-647.0,-647.0,-639.0,0.0
1223745,2396755,100003,Consumer loans,6737.310,68809.5,68053.5,6885.0,68809.5,SATURDAY,15,Y,1,0.100061,NaN,NaN,XAP,Approved,-2341,Cash through the bank,XAP,Family,Refreshed,Consumer Electronics,POS,XNA,Country-wide,200,Consumer electronics,12.0,middle,POS household with interest,365243.0,-2310.0,-1980.0,-1980.0,-1976.0,1.0


In [ ]:
previous_app.isnull().sum()/len(previous_app)*100

treshold=25
missing = (previous_app.isnull().sum()/len(previous_app))* 100
missing_cols = missing[missing > treshold].index
previous_app.drop(columns = missing_cols, inplace= True)
previous_app=previous_app.dropna()
previous_app.shape

(1246320, 26)

In [ ]:
previous_app.sample(3)

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION
1042730,1724256,121600,Consumer loans,13734.630,137362.5,123624.0,137362.5,FRIDAY,12,Y,1,XAP,Approved,-2384,XNA,XAP,Repeater,Consumer Electronics,POS,XNA,Stone,37,Consumer electronics,10.0,low_normal,POS household with interest
1026024,1007554,153259,Cash loans,40002.975,675000.0,767664.0,675000.0,WEDNESDAY,11,Y,1,XNA,Approved,-476,Cash through the bank,XAP,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,48.0,high,Cash X-Sell: high
1545845,1421114,217715,Consumer loans,5709.915,57105.0,51394.5,57105.0,THURSDAY,9,Y,1,XAP,Approved,-2675,Cashless from the account of the employer,XAP,Repeater,Consumer Electronics,POS,XNA,Stone,336,Consumer electronics,10.0,low_normal,POS household without interest


In [ ]:
# melakukan rename column
previous = previous_app.add_suffix('_pre')
previous.rename(columns={'SK_ID_PREV_pre':'SK_ID_PREV'},inplace=True)

# Drop kolom yang tidak diperlukan
previous.drop(columns=['WEEKDAY_APPR_PROCESS_START_pre','NAME_SELLER_INDUSTRY_pre','CODE_REJECT_REASON_pre','NAME_GOODS_CATEGORY_pre','NAME_CONTRACT_STATUS_pre','HOUR_APPR_PROCESS_START_pre','CHANNEL_TYPE_pre'],inplace=True)

previous.shape

(1246320, 19)

In [ ]:
# @title Join Table
previous_df =pd.merge(posh_card,previous, how='inner',on='SK_ID_PREV')
previous_df.drop(columns=['SK_ID_CURR_card','SK_ID_PREV_pos','SK_ID_CURR_pre','SK_DPD_card', 'SK_DPD_DEF_card'],inplace=True)
print('Jumlah Data',previous_df.shape)
previous_df.head()

Jumlah Data (172437, 46)


,SK_ID_PREV,SK_ID_CURR,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,NAME_CONTRACT_STATUS_card,AMT_BALANCE_card,AMT_CREDIT_LIMIT_ACTUAL_card,AMT_DRAWINGS_ATM_CURRENT_card,AMT_DRAWINGS_CURRENT_card,AMT_DRAWINGS_OTHER_CURRENT_card,AMT_DRAWINGS_POS_CURRENT_card,AMT_INST_MIN_REGULARITY_card,AMT_PAYMENT_CURRENT_card,AMT_PAYMENT_TOTAL_CURRENT_card,AMT_RECEIVABLE_PRINCIPAL_card,AMT_RECIVABLE_card,AMT_TOTAL_RECEIVABLE_card,CNT_DRAWINGS_ATM_CURRENT_card,CNT_DRAWINGS_CURRENT_card,CNT_DRAWINGS_OTHER_CURRENT_card,CNT_DRAWINGS_POS_CURRENT_card,CNT_INSTALMENT_MATURE_CUM_card,NAME_CONTRACT_STATUS_pos,CNT_INSTALMENT_pos,CNT_INSTALMENT_FUTURE_pos,SK_DPD_pos,SK_DPD_DEF_pos,NAME_CONTRACT_TYPE_pre,AMT_ANNUITY_pre,AMT_APPLICATION_pre,AMT_CREDIT_pre,AMT_GOODS_PRICE_pre,FLAG_LAST_APPL_PER_CONTRACT_pre,NFLAG_LAST_APPL_IN_DAY_pre,NAME_CASH_LOAN_PURPOSE_pre,DAYS_DECISION_pre,NAME_PAYMENT_TYPE_pre,NAME_CLIENT_TYPE_pre,NAME_PORTFOLIO_pre,NAME_PRODUCT_TYPE_pre,SELLERPLACE_AREA_pre,CNT_PAYMENT_pre,NAME_YIELD_GROUP_pre,PRODUCT_COMBINATION_pre
0,1000018,394447,-1332.0,-1338.0,35257.905,35257.905,Active,374731.425,405000,27000.0,147394.980,0.0,120394.980,12970.44,27708.75,27708.75,361490.985,368012.925,368012.925,6.0,44,0.0,38.0,10.0,Active,30.0,20.0,0,0,Revolving loans,2250.0,45000.0,45000.0,45000.0,Y,1,XAP,-176,XNA,New,Cards,walk-in,595,0.0,XNA,Card Street
1,1000030,361282,-2201.0,-2245.0,43398.495,43398.495,Active,447928.515,652500,4500.0,138059.505,0.0,133559.505,16625.79,43320.42,21263.58,443795.625,447483.015,447483.015,1.0,41,0.0,40.0,15.0,Active,30.0,20.0,0,0,Revolving loans,2250.0,45000.0,45000.0,45000.0,Y,1,XAP,-255,XNA,New,Cards,walk-in,100,0.0,XNA,Card Street
2,1000030,361282,-2201.0,-2245.0,43398.495,43398.495,Active,447928.515,652500,4500.0,138059.505,0.0,133559.505,16625.79,43320.42,21263.58,443795.625,447483.015,447483.015,1.0,41,0.0,40.0,15.0,Completed,5.0,0.0,0,0,Revolving loans,2250.0,45000.0,45000.0,45000.0,Y,1,XAP,-255,XNA,New,Cards,walk-in,100,0.0,XNA,Card Street
3,1000031,131335,-6419.0,-6542.0,354519.090,354519.090,Active,838311.030,2394000,157500.0,463353.840,0.0,305853.840,42804.81,354519.09,354519.09,822446.055,833599.530,833599.530,4.0,21,0.0,17.0,59.0,Active,30.0,27.0,0,0,Revolving loans,2250.0,45000.0,45000.0,45000.0,Y,1,XAP,-499,XNA,New,Cards,walk-in,150,0.0,XNA,Card Street
4,1000031,131335,-6419.0,-6542.0,354519.090,354519.090,Active,838311.030,2394000,157500.0,463353.840,0.0,305853.840,42804.81,354519.09,354519.09,822446.055,833599.530,833599.530,4.0,21,0.0,17.0,59.0,Active,144.0,78.0,0,0,Revolving loans,2250.0,45000.0,45000.0,45000.0,Y,1,XAP,-499,XNA,New,Cards,walk-in,150,0.0,XNA,Card Street


## Tabel TRAIN

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab minipro 4 /Permodelan data Home Credit Indonesia/Data HCI/application_train.csv')
print(f'Jumlah data Train: ',{train.shape})
print(f'Jumlah duplikat SK_ID_CURR: ',{train['SK_ID_CURR'].duplicated().sum()})

Jumlah data Train:  {(307511, 122)}
Jumlah duplikat SK_ID_CURR:  {0}


Berdasarkan kolom SK_ID_CURR pada tabel train terlihat bahwa setiap baris dari data mewakali 1 user. Tahap selanjutnya akan sama seperti tahap sebelumnya dari proses missing value hingga feature engineering.

### Handle Missing Value

In [ ]:
(train.isnull().sum()/len(train))*100

SK_ID_CURR                     0.000000
TARGET                         0.000000
NAME_CONTRACT_TYPE             0.000000
CODE_GENDER                    0.000000
FLAG_OWN_CAR                   0.000000
                                ...    
AMT_REQ_CREDIT_BUREAU_DAY     13.501631
AMT_REQ_CREDIT_BUREAU_WEEK    13.501631
AMT_REQ_CREDIT_BUREAU_MON     13.501631
AMT_REQ_CREDIT_BUREAU_QRT     13.501631
AMT_REQ_CREDIT_BUREAU_YEAR    13.501631
Length: 122, dtype: float64

In [ ]:
treshold = 10
train = drop_kolom(train, treshold)
cross_cek = train.isnull().sum()
cross_cek[cross_cek > 0]

AMT_ANNUITY                   12
AMT_GOODS_PRICE              278
NAME_TYPE_SUITE             1292
CNT_FAM_MEMBERS                2
EXT_SOURCE_2                 660
OBS_30_CNT_SOCIAL_CIRCLE    1021
DEF_30_CNT_SOCIAL_CIRCLE    1021
OBS_60_CNT_SOCIAL_CIRCLE    1021
DEF_60_CNT_SOCIAL_CIRCLE    1021
DAYS_LAST_PHONE_CHANGE         1
dtype: int64

In [ ]:
train[['AMT_ANNUITY','AMT_GOODS_PRICE','NAME_TYPE_SUITE','EXT_SOURCE_2','CNT_FAM_MEMBERS','DAYS_LAST_PHONE_CHANGE','OBS_30_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE','OBS_60_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE']].describe().T

,count,mean,std,min,25%,50%,75%,max
AMT_ANNUITY,307499.0,27108.573909,14493.737315,1.615500e+03,16524.000000,24903.000000,34596.000000,258025.500
AMT_GOODS_PRICE,307233.0,538396.207429,369446.460540,4.050000e+04,238500.000000,450000.000000,679500.000000,4050000.000
EXT_SOURCE_2,306851.0,0.514393,0.191060,8.173617e-08,0.392457,0.565961,0.663617,0.855
CNT_FAM_MEMBERS,307509.0,2.152665,0.910682,1.000000e+00,2.000000,2.000000,3.000000,20.000
DAYS_LAST_PHONE_CHANGE,307510.0,-962.858788,826.808487,-4.292000e+03,-1570.000000,-757.000000,-274.000000,0.000
OBS_30_CNT_SOCIAL_CIRCLE,306490.0,1.422245,2.400989,0.000000e+00,0.000000,0.000000,2.000000,348.000
DEF_30_CNT_SOCIAL_CIRCLE,306490.0,0.143421,0.446698,0.000000e+00,0.000000,0.000000,0.000000,34.000
OBS_60_CNT_SOCIAL_CIRCLE,306490.0,1.405292,2.379803,0.000000e+00,0.000000,0.000000,2.000000,344.000
DEF_60_CNT_SOCIAL_CIRCLE,306490.0,0.100049,0.362291,0.000000e+00,0.000000,0.000000,0.000000,24.000


In [ ]:
train['AMT_ANNUITY'] = train['AMT_ANNUITY'].fillna(train['AMT_ANNUITY'].median())
train['AMT_GOODS_PRICE'] = train['AMT_GOODS_PRICE'].fillna(train['AMT_GOODS_PRICE'].median())
train['EXT_SOURCE_2'] = train['EXT_SOURCE_2'].fillna(train['EXT_SOURCE_2'].median())
train['CNT_FAM_MEMBERS'] = train['CNT_FAM_MEMBERS'].fillna(train['CNT_FAM_MEMBERS'].median())
train['DAYS_LAST_PHONE_CHANGE'] = train['DAYS_LAST_PHONE_CHANGE'].fillna(train['DAYS_LAST_PHONE_CHANGE'].median())
train['NAME_TYPE_SUITE'] = train['NAME_TYPE_SUITE'].fillna(train['NAME_TYPE_SUITE'].mode()[0])
train['OBS_30_CNT_SOCIAL_CIRCLE'] = train['OBS_30_CNT_SOCIAL_CIRCLE'].fillna(train['OBS_30_CNT_SOCIAL_CIRCLE'].median())
train['DEF_30_CNT_SOCIAL_CIRCLE'] = train['DEF_30_CNT_SOCIAL_CIRCLE'].fillna(train['DEF_30_CNT_SOCIAL_CIRCLE'].median())
train['OBS_60_CNT_SOCIAL_CIRCLE'] = train['OBS_60_CNT_SOCIAL_CIRCLE'].fillna(train['OBS_60_CNT_SOCIAL_CIRCLE'].median())
train['DEF_60_CNT_SOCIAL_CIRCLE'] = train['DEF_60_CNT_SOCIAL_CIRCLE'].fillna(train['DEF_60_CNT_SOCIAL_CIRCLE'].median())

### Feature Engineering

In [ ]:
# mengabungkan kolom Flag Document 2- 19 diubah menjadi 1 feature
train['Sum_Document'] = train['FLAG_DOCUMENT_2'] + train['FLAG_DOCUMENT_3'] + train['FLAG_DOCUMENT_4'] + train['FLAG_DOCUMENT_5'] + train['FLAG_DOCUMENT_6'] + train['FLAG_DOCUMENT_7'] + train['FLAG_DOCUMENT_8'] + train['FLAG_DOCUMENT_9'] +train['FLAG_DOCUMENT_10'] + train['FLAG_DOCUMENT_11'] + train['FLAG_DOCUMENT_12'] + train['FLAG_DOCUMENT_13'] + train['FLAG_DOCUMENT_14'] + train['FLAG_DOCUMENT_15'] +train['FLAG_DOCUMENT_16'] + train['FLAG_DOCUMENT_17'] + train['FLAG_DOCUMENT_18'] + train['FLAG_DOCUMENT_19'] + train['FLAG_DOCUMENT_20'] + train['FLAG_DOCUMENT_21']
train.drop(columns=['FLAG_DOCUMENT_2','FLAG_DOCUMENT_3','FLAG_DOCUMENT_4','FLAG_DOCUMENT_5','FLAG_DOCUMENT_6','FLAG_DOCUMENT_7','FLAG_DOCUMENT_8','FLAG_DOCUMENT_9','FLAG_DOCUMENT_10','FLAG_DOCUMENT_11','FLAG_DOCUMENT_12','FLAG_DOCUMENT_13','FLAG_DOCUMENT_14','FLAG_DOCUMENT_15','FLAG_DOCUMENT_16','FLAG_DOCUMENT_17','FLAG_DOCUMENT_18','FLAG_DOCUMENT_19','FLAG_DOCUMENT_20','FLAG_DOCUMENT_21'],inplace=True)
train['STATUS_DOCUMENT'] = train['Sum_Document'].apply(kategori_dokument)
train.drop(columns='Sum_Document', inplace =True)

In [ ]:
# mengabungan kolom-kolom FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL menjadi 1 keterang bahwa user memiliki contact yang bisa dihubungi
train['ANY_ACTIVE_FLAG'] = train['FLAG_MOBIL'] + train['FLAG_EMP_PHONE'] + train['FLAG_WORK_PHONE'] + train['FLAG_CONT_MOBILE'] + train['FLAG_PHONE'] + train['FLAG_EMAIL']
train.drop(columns=['FLAG_MOBIL','FLAG_EMP_PHONE','FLAG_WORK_PHONE','FLAG_CONT_MOBILE','FLAG_PHONE','FLAG_EMAIL'],inplace=True)
train['ANY_ACTIVE_FLAG'] = train['ANY_ACTIVE_FLAG'].apply(information_flag).astype(int)

In [ ]:
# melakukan mapping untuk mengabungan kolom kepemilikan rumah dan mobil menjadi kolom kepemilikian aset (general)
train['FLAG_OWN_CAR'] = train['FLAG_OWN_CAR']. map({'Y': 1, 'N':0}).astype(int)
train['FLAG_OWN_REALTY'] = train['FLAG_OWN_REALTY']. map({'Y': 1, 'N':0}).astype(int)
train['Sum'] = train['FLAG_OWN_CAR'] + train['FLAG_OWN_REALTY']
train['INFORMATION_ASSET'] = train['Sum'].apply(information_flag)
train.drop(columns=['FLAG_OWN_CAR','FLAG_OWN_REALTY','Sum'],inplace =True)

In [ ]:
print('Sisa Data: ',train.shape)
train.columns

Sisa Data:  (307511, 40)


Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
       'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY',
       'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE',
       'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE',
       'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'CNT_FAM_MEMBERS',
       'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY',
       'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START',
       'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION',
       'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY',
       'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY',
       'ORGANIZATION_TYPE', 'EXT_SOURCE_2', 'OBS_30_CNT_SOCIAL_CIRCLE',
       'DEF_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE',
       'DEF_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE', 'STATUS_DOCUMENT',
       'ANY_ACTIVE_FLAG', 'INFORMATION_ASSET'],
      dt

In [ ]:
# @title Join table Train
Train_df = pd.merge(train, previous_df, how='inner',on='SK_ID_CURR')
print('Jumlah data: ',Train_df.shape)
Train_df.head()

Jumlah data:  (137887, 85)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_2,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,STATUS_DOCUMENT,ANY_ACTIVE_FLAG,INFORMATION_ASSET,SK_ID_PREV,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,NAME_CONTRACT_STATUS_card,AMT_BALANCE_card,AMT_CREDIT_LIMIT_ACTUAL_card,AMT_DRAWINGS_ATM_CURRENT_card,AMT_DRAWINGS_CURRENT_card,AMT_DRAWINGS_OTHER_CURRENT_card,AMT_DRAWINGS_POS_CURRENT_card,AMT_INST_MIN_REGULARITY_card,AMT_PAYMENT_CURRENT_card,AMT_PAYMENT_TOTAL_CURRENT_card,AMT_RECEIVABLE_PRINCIPAL_card,AMT_RECIVABLE_card,AMT_TOTAL_RECEIVABLE_card,CNT_DRAWINGS_ATM_CURRENT_card,CNT_DRAWINGS_CURRENT_card,CNT_DRAWINGS_OTHER_CURRENT_card,CNT_DRAWINGS_POS_CURRENT_card,CNT_INSTALMENT_MATURE_CUM_card,NAME_CONTRACT_STATUS_pos,CNT_INSTALMENT_pos,CNT_INSTALMENT_FUTURE_pos,SK_DPD_pos,SK_DPD_DEF_pos,NAME_CONTRACT_TYPE_pre,AMT_ANNUITY_pre,AMT_APPLICATION_pre,AMT_CREDIT_pre,AMT_GOODS_PRICE_pre,FLAG_LAST_APPL_PER_CONTRACT_pre,NFLAG_LAST_APPL_IN_DAY_pre,NAME_CASH_LOAN_PURPOSE_pre,DAYS_DECISION_pre,NAME_PAYMENT_TYPE_pre,NAME_CLIENT_TYPE_pre,NAME_PORTFOLIO_pre,NAME_PRODUCT_TYPE_pre,SELLERPLACE_AREA_pre,CNT_PAYMENT_pre,NAME_YIELD_GROUP_pre,PRODUCT_COMBINATION_pre
0,100048,0,Cash loans,F,0,202500.0,604152.0,29196.0,540000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.009175,-16971,-475,-3148.0,-513,2.0,2,2,MONDAY,12,0,1,1,0,1,1,Industry: type 1,0.037315,4.0,0.0,4.0,0.0,-458.0,incomplate,1,1,1060558,-26528.0,-26645.0,588664.890,588664.890,Active,1854127.260,6975000,27000.0,496296.00,0.0,469296.00,112413.375,586414.890,586414.890,1805438.565,1865730.465,1865730.465,1.0,8,0.0,7.0,358.0,Active,180.0,135.0,0,0,Revolving loans,11250.0,225000.0,225000.0,225000.0,Y,1,XAP,-949,XNA,New,Cards,walk-in,-1,0.0,XNA,Card Street
1,100048,0,Cash loans,F,0,202500.0,604152.0,29196.0,540000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.009175,-16971,-475,-3148.0,-513,2.0,2,2,MONDAY,12,0,1,1,0,1,1,Industry: type 1,0.037315,4.0,0.0,4.0,0.0,-458.0,incomplate,1,1,1060558,-26528.0,-26645.0,588664.890,588664.890,Active,1854127.260,6975000,27000.0,496296.00,0.0,469296.00,112413.375,586414.890,586414.890,1805438.565,1865730.465,1865730.465,1.0,8,0.0,7.0,358.0,Active,90.0,80.0,0,0,Revolving loans,11250.0,225000.0,225000.0,225000.0,Y,1,XAP,-949,XNA,New,Cards,walk-in,-1,0.0,XNA,Card Street
2,100048,0,Cash loans,F,0,202500.0,604152.0,29196.0,540000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.009175,-16971,-475,-3148.0,-513,2.0,2,2,MONDAY,12,0,1,1,0,1,1,Industry: type 1,0.037315,4.0,0.0,4.0,0.0,-458.0,incomplate,1,1,1060558,-26528.0,-26645.0,588664.890,588664.890,Active,1854127.260,6975000,27000.0,496296.00,0.0,469296.00,112413.375,586414.890,586414.890,1805438.565,1865730.465,1865730.465,1.0,8,0.0,7.0,358.0,Completed,5.0,0.0,0,0,Revolving loans,11250.0,225000.0,225000.0,225000.0,Y,1,XAP,-949,XNA,New,Cards,walk-in,-1,0.0,XNA,Card Street
3,100075,0,Cash loans,F,0,112500.0,95940.0,10462.5,90000.0,Family,Working,Higher education,Single / not married,House / apartment,0.004960,-12240,-3720,-5246.0,-4541,1.0,2,2,MONDAY,14,0,0,0,0,0,0,University,0.664544,0.0,0.0,0.0,0.0,-280.0,incomplate,1,0,1596605,-3612.0,-3673.0,582566.715,582566.715,Active,1308696.435,1800000,597375.0,599460.75,0.0,2085.75,50553.225,524066.715,524066.715,1160362.485,1215199.620,12151

## Tabel Test

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Colab minipro 4 /Permodelan data Home Credit Indonesia/Data HCI/application_test.csv')
print(f'Jumlah data Test: ',{test.shape})
print(f'Jumlah duplikat SK_ID_CURR: ',{test['SK_ID_CURR'].duplicated().sum()})
test.head(2)

Jumlah data Test:  {(48744, 121)}
Jumlah duplikat SK_ID_CURR:  {0}


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.018850,-19241,-2329,-5170.0,-812,NaN,1,1,0,1,0,1,NaN,2.0,2,2,TUESDAY,18,0,0,0,0,0,0,Kindergarten,0.752614,0.789654,0.159520,0.066,0.059,0.9732,NaN,NaN,NaN,0.1379,0.125,NaN,NaN,NaN,0.0505,NaN,NaN,0.0672,0.0612,0.9732,NaN,NaN,NaN,0.1379,0.125,NaN,NaN,NaN,0.0526,NaN,NaN,0.0666,0.059,0.9732,NaN,NaN,NaN,0.1379,0.125,NaN,NaN,NaN,0.0514,NaN,NaN,NaN,block of flats,0.0392,"Stone, brick",No,0.0,0.0,0.0,0.0,-1740.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.035792,-18064,-4469,-9118.0,-1623,NaN,1,1,0,1,0,0,Low-skill Laborers,2.0,2,2,FRIDAY,9,0,0,0,0,0,0,Self-employed,0.564990,0.291656,0.432962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0


Berdasarkan kolom SK_ID_CURR pada tabel test terlihat bahwa setiap baris dari data mewakali 1 user.

### Handle Missing Value Tabel Test

In [ ]:
test.isnull().sum()/len(test)*100

SK_ID_CURR                     0.000000
NAME_CONTRACT_TYPE             0.000000
CODE_GENDER                    0.000000
FLAG_OWN_CAR                   0.000000
FLAG_OWN_REALTY                0.000000
                                ...    
AMT_REQ_CREDIT_BUREAU_DAY     12.409732
AMT_REQ_CREDIT_BUREAU_WEEK    12.409732
AMT_REQ_CREDIT_BUREAU_MON     12.409732
AMT_REQ_CREDIT_BUREAU_QRT     12.409732
AMT_REQ_CREDIT_BUREAU_YEAR    12.409732
Length: 121, dtype: float64

In [ ]:
# Menghilangkan data yang memiliki nilai missing diatas 10%
treshold = 10
test = drop_kolom(test, treshold)
cross_cek = test.isnull().sum()
cross_cek[cross_cek > 0]

AMT_ANNUITY                  24
NAME_TYPE_SUITE             911
EXT_SOURCE_2                  8
OBS_30_CNT_SOCIAL_CIRCLE     29
DEF_30_CNT_SOCIAL_CIRCLE     29
OBS_60_CNT_SOCIAL_CIRCLE     29
DEF_60_CNT_SOCIAL_CIRCLE     29
dtype: int64

In [ ]:
# Melakukan analisa statistik pada feature - feature yang masih memiliki nilai missing
test[['AMT_ANNUITY','NAME_TYPE_SUITE','EXT_SOURCE_2','OBS_30_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE','OBS_60_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE']].describe().T

,count,mean,std,min,25%,50%,75%,max
AMT_ANNUITY,48720.0,29426.240209,16016.368315,2295.000000,17973.000000,26199.000000,37390.500000,180576.000
EXT_SOURCE_2,48736.0,0.518021,0.181278,0.000008,0.408066,0.558758,0.658497,0.855
OBS_30_CNT_SOCIAL_CIRCLE,48715.0,1.447644,3.608053,0.000000,0.000000,0.000000,2.000000,354.000
DEF_30_CNT_SOCIAL_CIRCLE,48715.0,0.143652,0.514413,0.000000,0.000000,0.000000,0.000000,34.000
OBS_60_CNT_SOCIAL_CIRCLE,48715.0,1.435738,3.580125,0.000000,0.000000,0.000000,2.000000,351.000
DEF_60_CNT_SOCIAL_CIRCLE,48715.0,0.101139,0.403791,0.000000,0.000000,0.000000,0.000000,24.000


In [ ]:
'''
Berdasarkan hasil diatas terlihat bahwa perbedaan atara nilai mean dan median cukup jauh, ini bisa dijadi kan indikator bahwa data memiliki
nilai outlier, dengan begitu saya akan memilih untuk menggunaka nilai median untuk mempertahakan distribusi datanya.
'''

test['AMT_ANNUITY'] = test['AMT_ANNUITY'].fillna(test['AMT_ANNUITY'].median())
test['EXT_SOURCE_2'] = test['EXT_SOURCE_2'].fillna(test['EXT_SOURCE_2'].median())
test['NAME_TYPE_SUITE'] = test['NAME_TYPE_SUITE'].fillna(test['NAME_TYPE_SUITE'].mode()[0])
test['OBS_30_CNT_SOCIAL_CIRCLE'] = test['OBS_30_CNT_SOCIAL_CIRCLE'].fillna(test['OBS_30_CNT_SOCIAL_CIRCLE'].median())
test['DEF_30_CNT_SOCIAL_CIRCLE'] = test['DEF_30_CNT_SOCIAL_CIRCLE'].fillna(test['DEF_30_CNT_SOCIAL_CIRCLE'].median())
test['OBS_60_CNT_SOCIAL_CIRCLE'] = test['OBS_60_CNT_SOCIAL_CIRCLE'].fillna(test['OBS_60_CNT_SOCIAL_CIRCLE'].median())
test['DEF_60_CNT_SOCIAL_CIRCLE'] = test['DEF_60_CNT_SOCIAL_CIRCLE'].fillna(test['DEF_60_CNT_SOCIAL_CIRCLE'].median())

### Feature Engineering

In [ ]:
# mengabungkan kolom Flag Document 2- 19 diubah menjadi 1 feature
test['Sum_Document'] = test['FLAG_DOCUMENT_2'] + test['FLAG_DOCUMENT_3'] + test['FLAG_DOCUMENT_4'] + test['FLAG_DOCUMENT_5'] + test['FLAG_DOCUMENT_6'] + test['FLAG_DOCUMENT_7'] + test['FLAG_DOCUMENT_8'] + test['FLAG_DOCUMENT_9'] +test['FLAG_DOCUMENT_10'] + test['FLAG_DOCUMENT_11'] + test['FLAG_DOCUMENT_12'] + test['FLAG_DOCUMENT_13'] + test['FLAG_DOCUMENT_14'] + test['FLAG_DOCUMENT_15'] +test['FLAG_DOCUMENT_16'] + test['FLAG_DOCUMENT_17'] + test['FLAG_DOCUMENT_18'] + test['FLAG_DOCUMENT_19'] + test['FLAG_DOCUMENT_20'] + test['FLAG_DOCUMENT_21']
test.drop(columns=['FLAG_DOCUMENT_2','FLAG_DOCUMENT_3','FLAG_DOCUMENT_4','FLAG_DOCUMENT_5','FLAG_DOCUMENT_6','FLAG_DOCUMENT_7','FLAG_DOCUMENT_8','FLAG_DOCUMENT_9','FLAG_DOCUMENT_10','FLAG_DOCUMENT_11','FLAG_DOCUMENT_12','FLAG_DOCUMENT_13','FLAG_DOCUMENT_14','FLAG_DOCUMENT_15','FLAG_DOCUMENT_16','FLAG_DOCUMENT_17','FLAG_DOCUMENT_18','FLAG_DOCUMENT_19','FLAG_DOCUMENT_20','FLAG_DOCUMENT_21'],inplace=True)
test['STATUS_DOCUMENT'] = test['Sum_Document'].apply(kategori_dokument)
test.drop(columns='Sum_Document', inplace=True)

In [ ]:
# mengabungan kolom-kolom FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL menjadi 1 keterang bahwa user memiliki contact yang bisa dihubungi
test['ANY_ACTIVE_FLAG'] = test['FLAG_MOBIL'] + test['FLAG_EMP_PHONE'] + test['FLAG_WORK_PHONE'] + test['FLAG_CONT_MOBILE'] + test['FLAG_PHONE'] + test['FLAG_EMAIL']
test.drop(columns=['FLAG_MOBIL','FLAG_EMP_PHONE','FLAG_WORK_PHONE','FLAG_CONT_MOBILE','FLAG_PHONE','FLAG_EMAIL'],inplace=True)

test['ANY_ACTIVE_FLAG'] = test['ANY_ACTIVE_FLAG'].apply(information_flag)
test['ANY_ACTIVE_FLAG'] = test['ANY_ACTIVE_FLAG'].astype(int)

In [ ]:
# melakukan mapping untuk mengabungan kolom kepemilikan rumah dan mobil menjadi kolom kepemilikian aset (general)
test['FLAG_OWN_CAR'] = test['FLAG_OWN_CAR']. map({'Y': 1, 'N':0}).astype(int)
test['FLAG_OWN_REALTY'] = test['FLAG_OWN_REALTY']. map({'Y': 1, 'N':0}).astype(int)
test['Sum'] = test['FLAG_OWN_CAR'] + test['FLAG_OWN_REALTY']

test['INFORMATION_ASSET'] = test['Sum'].apply(information_flag)
test.drop(columns=['FLAG_OWN_CAR','FLAG_OWN_REALTY','Sum'],inplace =True)

In [ ]:
# melihat hasil data setelah dilakukan data cleaning.
print('Data yang dimiliki: ', test.shape)
test.columns

Data yang dimiliki:  (48744, 39)


Index(['SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'CNT_CHILDREN',
       'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
       'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'REGION_POPULATION_RELATIVE',
       'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH',
       'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT',
       'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START',
       'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION',
       'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION',
       'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY',
       'LIVE_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE', 'EXT_SOURCE_2',
       'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE',
       'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE',
       'DAYS_LAST_PHONE_CHANGE', 'STATUS_DOCUMENT', 'ANY_ACTIVE_FLAG',
       'INFORMATION_ASSET'],
      dtype='objec

In [ ]:
# @title Join Table Test
Test_df = pd.merge(test, previous_df, how='inner',on='SK_ID_CURR')
print('Jumlah data: ',Test_df.shape)
Test_df.head()

Jumlah data:  (34550, 84)


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_2,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,STATUS_DOCUMENT,ANY_ACTIVE_FLAG,INFORMATION_ASSET,SK_ID_PREV,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,NAME_CONTRACT_STATUS_card,AMT_BALANCE_card,AMT_CREDIT_LIMIT_ACTUAL_card,AMT_DRAWINGS_ATM_CURRENT_card,AMT_DRAWINGS_CURRENT_card,AMT_DRAWINGS_OTHER_CURRENT_card,AMT_DRAWINGS_POS_CURRENT_card,AMT_INST_MIN_REGULARITY_card,AMT_PAYMENT_CURRENT_card,AMT_PAYMENT_TOTAL_CURRENT_card,AMT_RECEIVABLE_PRINCIPAL_card,AMT_RECIVABLE_card,AMT_TOTAL_RECEIVABLE_card,CNT_DRAWINGS_ATM_CURRENT_card,CNT_DRAWINGS_CURRENT_card,CNT_DRAWINGS_OTHER_CURRENT_card,CNT_DRAWINGS_POS_CURRENT_card,CNT_INSTALMENT_MATURE_CUM_card,NAME_CONTRACT_STATUS_pos,CNT_INSTALMENT_pos,CNT_INSTALMENT_FUTURE_pos,SK_DPD_pos,SK_DPD_DEF_pos,NAME_CONTRACT_TYPE_pre,AMT_ANNUITY_pre,AMT_APPLICATION_pre,AMT_CREDIT_pre,AMT_GOODS_PRICE_pre,FLAG_LAST_APPL_PER_CONTRACT_pre,NFLAG_LAST_APPL_IN_DAY_pre,NAME_CASH_LOAN_PURPOSE_pre,DAYS_DECISION_pre,NAME_PAYMENT_TYPE_pre,NAME_CLIENT_TYPE_pre,NAME_PORTFOLIO_pre,NAME_PRODUCT_TYPE_pre,SELLERPLACE_AREA_pre,CNT_PAYMENT_pre,NAME_YIELD_GROUP_pre,PRODUCT_COMBINATION_pre
0,100107,Cash loans,M,0,180000.0,296280.0,21690.0,225000.0,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.032561,-13563,-1007,-5719.0,-4044,2.0,1,1,TUESDAY,15,0,0,0,0,0,0,Business Entity Type 3,0.684407,2.0,0.0,2.0,0.0,-317.0,incomplate,1,1,1128717,-3951.0,-3934.0,126988.38,113395.815,Active,826106.895,1125000,0.0,206499.42,0.0,206499.42,36091.035,97701.255,70769.79,810294.21,824320.395,824320.395,0.0,82,0.0,82.0,36.0,Active,120.0,75.0,0,0,Revolving loans,2250.0,45000.0,45000.0,45000.0,Y,1,XAP,-318,XNA,New,Cards,walk-in,100,0.0,XNA,Card Street
1,100107,Cash loans,M,0,180000.0,296280.0,21690.0,225000.0,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.032561,-13563,-1007,-5719.0,-4044,2.0,1,1,TUESDAY,15,0,0,0,0,0,0,Business Entity Type 3,0.684407,2.0,0.0,2.0,0.0,-317.0,incomplate,1,1,1128717,-3951.0,-3934.0,126988.38,113395.815,Active,826106.895,1125000,0.0,206499.42,0.0,206499.42,36091.035,97701.255,70769.79,810294.21,824320.395,824320.395,0.0,82,0.0,82.0,36.0,Completed,10.0,0.0,0,0,Revolving loans,2250.0,45000.0,45000.0,45000.0,Y,1,XAP,-318,XNA,New,Cards,walk-in,100,0.0,XNA,Card Street
2,100169,Cash loans,F,0,135000.0,540000.0,27702.0,540000.0,Family,Pensioner,Higher education,Separated,House / apartment,0.072508,-22784,365243,-5103.0,-5229,1.0,1,1,TUESDAY,12,0,0,0,0,0,0,XNA,0.638614,0.0,0.0,0.0,0.0,-519.0,incomplate,1,1,2234740,-2245.0,-2254.0,242241.75,242241.750,Active,238705.065,3348000,225000.0,225000.00,0.0,0.00,14448.195,242241.750,239991.75,225000.00,229309.065,229309.065,4.0,4,0.0,0.0,38.0,Active,132.0,77.0,0,0,Revolving loans,20250.0,405000.0,405000.0,405000.0,Y,1,XAP,-446,XNA,Repeater,Cards,x-sell,-1,0.0,XNA,Card X-Sell
3,100169,Cash loans,F,0,135000.0,540000.0,27702.0,540000.0,Family,Pensioner,Higher education,Separated,House / apartment,0.072508,-22784,365243,-5103.0,-5229,1.0,1,1,TUESDAY,12,0,0,0,0,0,0,XNA,0.638614,0.0,0.0,0.0,0.0,-519.0,incomplate,1,1,2234740,-2245.0,-2254.0,242241.75,242241.750,Active,238705.065,3348000,225000.0,225000.00,0.0,0.00,14448.195,242241.750,239991.75,225000.00,229309.065,229309.065,4.0,4,0.0,0.0,38.0,Completed,11.0,0.0,0,0,Revolving loans,20250.0,405

In [ ]:
Train_df.to_csv('Train.csv', index=False)
# Test_df.to_csv('Test.csv', index=False)